## Libs

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
#from sklearn.metrics import 

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
from functools import partial
from itertools import chain

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

%matplotlib inline
np.random.seed(1)

-----
## Train, valid split

In [ ]:
pars_smp_train = np.load('data/pars_smp_train.npy')
y_smp_train = np.load('data/y_smp_train.npy')

In [ ]:
pars_smp_train.shape, y_smp_train.shape

In [ ]:
test_data_size = 1000000
small_pars_smp_train = pars_smp_train[:test_data_size].copy()
small_y_smp_train = y_smp_train[:test_data_size].copy()

In [ ]:
if test_data_size != 1000000:
    X_train, X_valid, y_train, y_valid = train_test_split(small_pars_smp_train, small_y_smp_train, test_size=0.2, shuffle=False, random_state=178)
else:
   X_train=small_pars_smp_train
   y_train=small_y_smp_train
   X_valid = np.array(0)
   y_valid = np.array(0)

In [ ]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
type(small_y_smp_train)

In [ ]:
type(X_train)

In [ ]:
use_cuda = True
if use_cuda:
    device = 'cuda'
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    device = 'cpu'

In [ ]:
torch.cuda.is_available()

## Build the Model

In [ ]:
class q_model(nn.Module):
    def __init__(self, 
                 quantiles, 
                 in_shape=1,  
                 dropout=0.5):     
        super().__init__()
        self.quantiles = quantiles
        self.num_quantiles = len(quantiles)
        
        self.in_shape = in_shape
        self.out_shape = len(quantiles)
        self.dropout = dropout
        self.build_model()
        self.init_weights()
        
    def build_model(self): 
        self.base_model = nn.LSTM(3, 128, 2, batch_first=True, dropout=0.1)#input in 3 
        final_layers = [
            nn.Linear(128, 15) for _ in range(len(self.quantiles))#output in 15 
        ]
        self.final_layers = nn.ModuleList(final_layers)
        
    def init_weights(self):
        for m in chain(self.final_layers):
            if isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight)
                nn.init.constant_(m.bias, 0)        
    
    def forward(self, x):
        out, _ = self.base_model(x)
        return torch.stack([layer(out[:, -1, :]) for layer in self.final_layers], dim=1)

In [ ]:
class QuantileLoss(nn.Module):
    def __init__(self, quantiles):
        super().__init__()
        self.quantiles = quantiles
        
    def forward(self, preds, target):
        assert not target.requires_grad
        assert preds.size(0) == target.size(0)
        losses = []
        for i, q in enumerate(self.quantiles):
            errors = target - preds[:, i]
            losses.append(torch.max((q-1) * errors, q * errors).unsqueeze(1))
        loss = torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))
        return loss

In [ ]:
import tqdm
class Learner:
    def __init__(self, model, optimizer_class, loss_func, device='cpu'):
        self.model = model.to(device)
        self.optimizer = optimizer_class(self.model.parameters())
        self.loss_func = loss_func.to(device)
        self.device = device
        self.loss_history = []
        
    def fit(self, x, y, epochs, batch_size):
        self.model.train()
        for e in tqdm.tqdm(range(epochs)):
            shuffle_idx = np.arange(x.shape[0])
            np.random.shuffle(shuffle_idx)
            x = x[shuffle_idx]
            y = y[shuffle_idx]
            epoch_losses = []
            for idx in range(0, x.shape[0], batch_size):
                self.optimizer.zero_grad()
                batch_x = torch.from_numpy(
                    x[idx : min(idx + batch_size, x.shape[0]),:]
                ).float().to(self.device).requires_grad_(False)
                #print(batch_x)
                #print(type(batch_x))
                batch_y = torch.from_numpy(
                    y[idx : min(idx + batch_size, y.shape[0])]
                ).float().to(self.device).requires_grad_(False)
                preds = self.model(batch_x)
                loss = self.loss_func(preds, batch_y)
                loss.backward()
                self.optimizer.step()
                epoch_losses.append(loss.cpu().detach().numpy())                                
            epoch_loss =  np.mean(epoch_losses)
            self.loss_history.append(epoch_loss)
            print("Epoch {}: {}".format(e+1, epoch_loss))
                
    def predict(self, x, mc=False):
        if mc:
            self.model.train()
        else:
            self.model.eval()
        return self.model(torch.from_numpy(x).float().to(self.device).requires_grad_(False)).cpu().detach().numpy()

## Setup Learner class

In [ ]:
# Instantiate model
quantiles = [.1, .25, .50,.75,.90]
model = q_model(quantiles, dropout=0.1)
loss_func = QuantileLoss(quantiles)
learner = Learner(model, partial(torch.optim.Adam, weight_decay=1e-6, lr=5e-3), loss_func, device=device)

## Train the Model

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
X_train.squeeze(2).shape

In [ ]:
# Run training
epochs = 15
learner.fit(y_train, X_train.squeeze(2), epochs, batch_size=200)

In [ ]:
def upd_lr(optimizer_class, learner):
    learner.optimizer = optimizer_class(learner.model.parameters())
    return learner
learner2 = upd_lr(partial(torch.optim.Adam, weight_decay=1e-7, lr=5e-4), learner)

In [ ]:
learner2.optimizer

In [ ]:
epochs = 15
learner2.fit(y_train, X_train.squeeze(2), epochs, batch_size=500)

In [ ]:
import pickle
filename = "learner_128_lstm2_lr5e-4_tune6.class"
file = open(filename, 'wb') 
pickle.dump(learner2, file=file)#check correct learner!!!!


## Load data

In [ ]:
y_smp_test = np.load('data/y_smp_test.npy')

In [ ]:
y_smp_test.shape

In [ ]:
y_test = torch.Tensor(y_smp_test)
test_dataset = TensorDataset(y_test)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [ ]:
import pickle
filename = "learner_128_lstm2_lr5e-4_tune6.class"
filehandler = open(filename, 'rb') 
learner_load = pickle.load(filehandler)

### predict

In [ ]:
with torch.no_grad():
    all_output_test = []
    for batch in test_loader:
        inputs_test= batch[0]
        tmp = learner_load.predict(inputs_test.cpu().numpy())
        all_output_test.append(tmp)
    final_output_test = np.concatenate(all_output_test)

In [ ]:
final_output_test.shape

In [ ]:
final = np.transpose(final_output_test, (0, 2, 1))

In [ ]:
final2 = final.copy()
new_column = final2[:, :, 2]
arr_with_new_column = np.insert(final2, 0, new_column, axis=2)
arr_with_new_column.shape

In [ ]:
arr_with_new_column[0][0]

### save submit to file

In [ ]:
np.save(file='submission_mean_eq_50quantile_model_128_lstm2_lr5e-3_tune6.class.npy', arr=arr_with_new_column)